# Tune Dropout rate

In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


import tensorflow as tf
import keras

num_classes = 25

In [3]:
def create_model(dropout_rate=.0):
  input = tf.keras.Input(shape=(64,64,3))

  # Block 1
  x = tf.keras.layers.Conv2D(128, kernel_size=(3 ,3), activation='relu', strides=(2, 2), padding="same")(input)
  x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)

  x = tf.keras.layers.Conv2D(64, kernel_size=(4 ,4), activation='relu', padding="valid")(x)
  x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1, 1))(x)

  x = tf.keras.layers.Dropout(dropout_rate)(x)
  x = tf.keras.layers.Flatten()(x)

  x = tf.keras.layers.Dense(50, activation='relu')(x)
  output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)


  # bind all
  model = tf.keras.Model(input, output)

  model.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics=["accuracy"]
  )
  return model

In [4]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [5]:
X = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/imgs.npy')
Y = np.load('/content/drive/MyDrive/Colab Notebooks/cnn_img_avril/labels.npy')

In [6]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.757391 using {'dropout_rate': 0.1}
0.593233 (0.194655) with: {'dropout_rate': 0.0}
0.757391 (0.053165) with: {'dropout_rate': 0.1}
0.691643 (0.064900) with: {'dropout_rate': 0.2}
0.576796 (0.099906) with: {'dropout_rate': 0.3}
0.541360 (0.160816) with: {'dropout_rate': 0.4}
0.518092 (0.151903) with: {'dropout_rate': 0.5}
0.478386 (0.022167) with: {'dropout_rate': 0.6}
0.452450 (0.043602) with: {'dropout_rate': 0.7}
0.359377 (0.008971) with: {'dropout_rate': 0.8}
0.338350 (0.006144) with: {'dropout_rate': 0.9}
